<a href="https://colab.research.google.com/github/ayushijainn111/GenAI-APIs/blob/main/Crio_Do_Meal_Suggestion_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
# pip install --upgrade google-generativeai

###  1.Users should understand the need for using APIs to access LLMs

#### 1.1 Stimulate a traditional assistant without AI

In [30]:
#NOTE:
# random is a library used in python to generate random numbers or make random choices

import random

meals = [
    "Avocado toast with egg",
    "Grilled chicken and veggies",
    "Fruit salad bowl"
]

# random.choice() is used to pick one item at random from a list or other sequence

print("Meal suggestion:", random.choice(meals))

Meal suggestion: Grilled chicken and veggies


#### 1.2Stimulate Local AI (which is not possible)

In [31]:
# Requirements to construct a local LLM:
# 100+ GB of RAM
# 10+ GB model download (like LLaMA)
# High-end GPUs
# Complex installation & optimization

# This is to understand the importance of the need to use API’s to access LLMs.

### 2.Users should be able to implement best practices for API error handling and retries

#### 2.1Function with retries and exponential backoff to handle API errors

In [32]:
import time

def safe_gemini_call(prompt, retries=3):
    for attempt in range(retries):

    # try - except Block is used to catch and handle errors
        try:
            response = model.generate_content(prompt)
            return response.text.strip()

    #Exception as e captures the error message into a variable e
        except Exception as e:
            print(f"Attempt {attempt+1} failed: {e}")
    # time.sleep() pauses the program for given no of seconds
    #before continuing
            time.sleep(2 ** attempt)
    return "Sorry, couldn't generate a response."

### 3.Users should be able to Authenticate and interact with GenAI APIs to generate text, audio, and image

#### 3.1Code to make basic Gemini API to extract text

In [33]:
# to install Google’s Generative AI python library, allowing developers to use AI models

# pip install google-generativeai


import google.generativeai as genai

# configure is used to set up authentication by providing API key to access AI models
genai.configure(api_key="YOUR_API_KEY")

#.GenerativeModel create instance of a specific Google AI model
model = genai.GenerativeModel("gemini-2.0-flash")

# .generate_content is a method used to send a prompt
response = model.generate_content("Name one meal I can eat today.No explanation")

meal= response

print("Meal Suggestion:", meal)

Meal Suggestion: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Pasta salad\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.573649009068807
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 10,
        "candidates_token_count": 3,
        "total_token_count": 13
      },
      "model_version": "gemini-2.0-flash"
    }),
)


In [34]:
# .text is used to access the content of a response as a plain string
#.strip() is used to remove any leading and trailing whitespace
meal = response.text.strip()

print("\n Meal Suggestion:",meal)


 Meal Suggestion: Pasta salad


#### 3.2Code to make basic HuggingFace API to generate image

In [35]:
# requests: Simplifies sending HTTP requests like GET and POST in Python.
# PIL : Used for opening, editing, and saving images.
# BytesIO : Handles byte data as a file-like object in memory.

import requests
from PIL import Image
from io import BytesIO

# Hugging Face API key and endpoint for Stable Diffusion model
HF_API_KEY = "YOUR_HUGGINGFACE_API_KEY"
API_URL = "https://api-inference.huggingface.co/models/runwayml/stable-diffusion-v1-5"


#Sets headers for authentication and specifies the request body type as JSON format.
headers = {
    "Authorization": f"Bearer {HF_API_KEY}",
    "Content-Type": "application/json"
}

user_prompt = meal

# Prepare the payload with the user prompt
payload = {
    "inputs": user_prompt,
    "options": {"wait_for_model": True}
}

# Send request to Hugging Face API
print("\n Generating image...")

response = requests.post(API_URL, headers=headers, json=payload)

if response.status_code == 200:
    # Open the generated image from the response
    image_data = response.content
    image = Image.open(BytesIO(image_data))

    # Display the image
    image.show()

    # Optional: Save the image
    image.save("generated_image.png")
    print(" Image saved as generated_image.png")
else:
    print(" Image generation failed:", response.status_code)
    print(response.text)


 Generating image...
 Image saved as generated_image.png


#### 3.3Code to make basic gTTs generate audio

In [36]:
# pip install gTTs

In [37]:
# pip install gTTs

import google.generativeai as genai
from gtts import gTTS
import os

# Configure Gemini

genai.configure(api_key="YOUR_API_KEY")

prompt = f"Explain why {meal} is beneficial for breakfast in one simple paragraph, without using bullet points or lists.."

# .text is used to access the content of a response as a plain string
#.strip() is used to remove any leading and trailing whitespace

response = model.generate_content(prompt)
formatted_explanation = response.text.strip()

print("Explanation:", formatted_explanation)


# Convert the generated explanation to audio using gTTS

def generate_audio(formatted_explanation):
    # Generate audio using gTTS
    tts = gTTS(text=formatted_explanation, lang='en')
     # Save the audio to a file
    tts.save("meal_suggestion.mp3")
    print("Audio saved as 'meal_suggestion.mp3'")

# Generate and save audio from the quote

generate_audio(formatted_explanation)

Explanation: Pasta salad can be a surprisingly beneficial breakfast option because it offers a balanced combination of carbohydrates for sustained energy, protein for satiety and muscle support (depending on the additions like cheese, meat, or beans), and fiber from vegetables, contributing to improved digestion and preventing blood sugar spikes early in the day. This nutritional profile can fuel your morning activities and keep you feeling full and focused until your next meal, making it a more substantial and potentially healthier alternative to sugary cereals or pastries.
Audio saved as 'meal_suggestion.mp3'


### 4.Users should be able to understand the need for parsing and utilizing structured API responses.

#### 4.1Use Spoonacular API to parse the suggested meal to extract the recipe title and ID

In [38]:
import requests

api_key = "YOUR_SPOONACULAR_API_KEY"
meal_name = meal

url = "https://api.spoonacular.com/recipes/complexSearch"
params = {
    "query": meal_name,
    "number": 1,
    "apiKey": api_key
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    results = data.get("results", [])
    if results:
        recipe = results[0]
        print(f"Recipe Title: {recipe['title']}")
        print(f"Recipe ID: {recipe['id']}")
    else:
        print("No recipes found for this meal.")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Recipe Title: Tricolor Rotini pasta Salad
Recipe ID: 663813


#### 4.2Use the recipe title generated to explain the dish in a structured way

In [39]:
import google.generativeai as genai

# Configure with your Gemini API key
genai.configure(api_key="YOUR_API_KEY")

# Initialize the model
model = genai.GenerativeModel("gemini-2.0-flash")

# Use recipe title to generate a descriptive explanation
prompt = f"Explain what the meal '{recipe['title']}' is, and how it's typically prepared."

response = model.generate_content(prompt)

#  Parse the structured GenAI response
genai_text = response.text
print(genai_text)

Tricolor Rotini Pasta Salad is a classic and versatile dish, popular for potlucks, picnics, and easy weeknight meals. It's essentially a cold pasta salad featuring rotini pasta that comes in three colors (hence the "tricolor").  These colors are usually red (tomato), green (spinach), and plain/white (semolina).

Here's a breakdown of what it is and how it's typically prepared:

**What it is:**

*   **Base:** Rotini pasta, distinguished by its corkscrew shape and the three colors. The shape holds the dressing and ingredients well.
*   **Ingredients:** Beyond the pasta, the ingredients are highly customizable. Common additions include:
    *   **Vegetables:**  Cherry tomatoes (halved or quartered), bell peppers (diced), cucumbers (diced), red onion (thinly sliced or diced), black olives (sliced), broccoli florets (blanched or raw), carrots (shredded or diced), celery (diced), zucchini (diced).
    *   **Cheese:**  Cubes of mozzarella, provolone, cheddar, or feta. Sometimes Parmesan chees

### 5.Users should be able to integrate multiple APIs into a single workflow

#### 5.1Use Spoonacular API to generate the ingredients required for the meal

In [40]:
import requests

api_key = "YOUR_SPOONACULAR_API_KEY"
recipe_id = recipe['id']  # Replace with the actual ID from complexSearch

url = f"https://api.spoonacular.com/recipes/{recipe_id}/information"

params = {
    "apiKey": api_key,
    "includeNutrition": False  # Optional
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    print(f"Title: {data['title']}")
    print("Ingredients:")
    for ing in data["extendedIngredients"]:
        print(f"- {ing['original']}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Title: Tricolor Rotini pasta Salad
Ingredients:
- 1) Tri colour Rotini Pasta 100 gm
- 2) Lettuce 2 Leaves
- 3) Olives 10 nos
- 4) Cucumber 1 small
- 5) Plum tomato 5 nos
- 6) Mayonnaise 2 Tbsp
- 7) Black pepper powder 1 Tsp
- 8) Salt ¼ Tsp


#### 5.2Extract the nutrition of the ingredients using recipe id

In [41]:
import requests

api_key = "YOUR_SPOONACULAR_API_KEY"
recipe_id = recipe['id']

# --- Get Recipe Info ---
info_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information"
info_params = {
    "apiKey": api_key,
    "includeNutrition": False
}

info_response = requests.get(info_url, params=info_params)
# --- Get Nutrition Info ---
nutrition_url = f"https://api.spoonacular.com/recipes/{recipe_id}/nutritionWidget.json"
nutrition_params = {
    "apiKey": api_key
}

nutrition_response = requests.get(nutrition_url, params=nutrition_params)

if nutrition_response.status_code == 200:
    nutrition_data = nutrition_response.json()
    print("\n Nutrition Info:")
    for nutrient in nutrition_data["nutrients"]:
        print(f"- {nutrient['name']}: {nutrient['amount']} {nutrient['unit']} ({nutrient['percentOfDailyNeeds']}% of daily needs)")
else:
    print(f"\nError fetching nutrition data: {nutrition_response.status_code}")
    print(nutrition_response.text)



 Nutrition Info:
- Calories: 145.59 kcal (7.28% of daily needs)
- Fat: 5.04 g (7.76% of daily needs)
- Saturated Fat: 0.71 g (4.44% of daily needs)
- Carbohydrates: 22.31 g (7.44% of daily needs)
- Net Carbohydrates: 14.85 g (5.4% of daily needs)
- Sugar: 13.02 g (14.47% of daily needs)
- Cholesterol: 1.26 mg (0.42% of daily needs)
- Sodium: 1690.52 mg (73.5% of daily needs)
- Alcohol: 0 g (100% of daily needs)
- Alcohol %: 0 % (100% of daily needs)
- Protein: 5.72 g (11.44% of daily needs)
- Vitamin A: 4981.99 IU (99.64% of daily needs)
- Vitamin K: 63.23 µg (60.22% of daily needs)
- Vitamin C: 47.43 mg (57.49% of daily needs)
- Manganese: 0.77 mg (38.27% of daily needs)
- Potassium: 1304.95 mg (37.28% of daily needs)
- Folate: 120.5 µg (30.13% of daily needs)
- Fiber: 7.47 g (29.86% of daily needs)
- Vitamin B6: 0.54 mg (27.13% of daily needs)
- Copper: 0.54 mg (27.05% of daily needs)
- Magnesium: 98.49 mg (24.62% of daily needs)
- Vitamin B1: 0.28 mg (18.72% of daily needs)
- Phosp